In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
import numpy as np
import random

In [ ]:
# Siamese Network
class SiameseNetwork(nn.Module):
    def __init__(self):
        super(SiameseNetwork, self).__init__()
        self.convnet = nn.Sequential(
            nn.Conv2d(1, 32, 5),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(32, 64, 5),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2)
        )

        self.fc = nn.Sequential(
            nn.Linear(1024, 256),
            nn.ReLU(inplace=True),
            nn.Linear(256, 256),
            nn.ReLU(inplace=True),
            nn.Linear(256, 2)
        )

    def forward_one(self, x):
        x = self.convnet(x)
        x = x.view(x.size()[0], -1)
        x = self.fc(x)
        return x

    def forward(self, input1, input2):
        output1 = self.forward_one(input1)
        output2 = self.forward_one(input2)
        return output1, output2

In [ ]:
# Contrastive Loss
class ContrastiveLoss(nn.Module):
    def __init__(self, margin=2.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, output1, output2, label):
        euclidean_distance = nn.functional.pairwise_distance(output1, output2)
        loss_contrastive = torch.mean((1-label) * torch.pow(euclidean_distance, 2) +
                                      (label) * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2))
        return loss_contrastive

In [ ]:
# Siamese MNIST Dataset
class SiameseMNIST(Dataset):
    def __init__(self, mnist_dataset):
        self.mnist_dataset = mnist_dataset
        self.labels = mnist_dataset.targets
        self.data = mnist_dataset.data
        self.label_to_indices = {label: np.where(self.labels.numpy() == label)[0]
                                 for label in np.unique(self.labels.numpy())}

    def __getitem__(self, index):
        target = np.random.randint(0, 2)
        img1, label1 = self.data[index], self.labels[index].item()
        if target == 1:
            siamese_index = index
            while siamese_index == index:
                siamese_index = np.random.choice(self.label_to_indices[label1])
        else:
            siamese_label = np.random.choice(list(set(range(10)) - set([label1])))
            siamese_index = np.random.choice(self.label_to_indices[siamese_label])
        img2 = self.data[siamese_index]

        # Ensure images are in the correct format (C, H, W)
        img1 = img1.unsqueeze(0).float() / 255.0  # MNIST images are grayscale, so 1 channel
        img2 = img2.unsqueeze(0).float() / 255.0

        return img1, img2, torch.from_numpy(np.array([target], dtype=np.float32))

    def __len__(self):
        return len(self.mnist_dataset)


In [ ]:
# Loading the MNIST dataset
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transforms.ToTensor())

# Creating the SiameseMNIST dataset
siamese_train_dataset = SiameseMNIST(train_dataset)
siamese_test_dataset = SiameseMNIST(test_dataset)

# Data loaders
train_loader = DataLoader(siamese_train_dataset, shuffle=True, batch_size=1024)
test_loader = DataLoader(siamese_test_dataset, shuffle=False, batch_size=1024)


In [ ]:
def train(model, criterion, optimizer, num_epochs=1):
    model.train()
    for epoch in range(num_epochs):
        for data in train_loader:
            img1, img2, target = data
            img1 = img1.to("cuda")
            img2 = img2.to("cuda")
            target = target.to("cuda")
            optimizer.zero_grad()
            output1, output2 = model(img1, img2)
            loss = criterion(output1, output2, target)
            loss.backward()
            optimizer.step()
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')



# Initialize network, criterion, and optimizer
net = SiameseNetwork().to("cuda")
criterion = ContrastiveLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

# Train and evaluate the model
train(net, criterion, optimizer, num_epochs=5)



Epoch [1/5], Loss: 1.2084
Epoch [2/5], Loss: 1.2107
Epoch [3/5], Loss: 1.1760
Epoch [4/5], Loss: 1.2247
Epoch [5/5], Loss: 1.2306
